### Fill In Missing 

In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from pyspark.sql.functions import pandas_udf, PandasUDFType
import pathlib
import time
import pyspark.pandas as ps

import numpy as np

from datetime import date, datetime, timedelta
from pyspark.pandas.indexes.datetimes import DatetimeIndex
import pyspark
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import col,isnan,when,count, when, col, rank, lit, monotonically_increasing_id, date_trunc, udf, min, max, explode
from pyspark.sql.types import StructType, StructField, TimestampType, IntegerType, FloatType, DateType, StringType

In [2]:
conf = pyspark.SparkConf().setAll([\
    ('spark.app.name', 'ReduceData')])
spark = SparkSession.builder.config(conf=conf)\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/23 01:55:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
glucose_data_schema=StructType([StructField('NumId', IntegerType(), True),
                                        StructField('PatientId', StringType(), True),
                                        StructField('Value', FloatType(), True),
                                        StructField('GlucoseDisplayTime', TimestampType(), True),
                                        StructField('GlucoseDisplayTimeRaw', StringType(), True),
                                        StructField('GlucoseDisplayDate', DateType(), True)])

In [8]:
read_in_types = ['training_set', 'test_set', 'val_set']
data_types = ['train', 'test', 'val']

# for data_type in data_types: 
for j in range(0,3): 
    data_location = "/cephfs/train_test_val/" + read_in_types[j]
    allPaths = [str(x) for x in list(pathlib.Path(data_location).glob('*.parquet')) if 'part-00' in str(x)]
    path_counter = 0
    
    for path in allPaths:
        gluc = pd.read_parquet(path, columns=['NumId','GlucoseDisplayTime', 'Value'])
        gluc['GlucoseDisplayTime'] = gluc['GlucoseDisplayTime'].dt.floor('Min')
        gluc = gluc.sort_values(by=['NumId', 'GlucoseDisplayTime'])

        min_max = gluc.groupby('NumId')\
                        .agg({'GlucoseDisplayTime' : ['min','max']})

        merge_df = pd.DataFrame(columns=['GlucoseDisplayTime', 'NumId'])
        starttime = time.time()
        last_idx = len(min_max)-1

        index_counter = 0
        for idx, row in min_max.iterrows():
            #grab all potential dates in range

            min_val = row['GlucoseDisplayTime']['min']
            max_val = row['GlucoseDisplayTime']['max']

            date_df = pd.DataFrame(pd.date_range(min_val, max_val, freq='5min'),\
                                   columns=['GlucoseDisplayTime'])  

            # merge dates with big pypsark df
            id_df = gluc[gluc['NumId'] == idx]

            mean = id_df.Value.mean().round()

            id_df.set_index('GlucoseDisplayTime', inplace=True)    

            date_df.set_index('GlucoseDisplayTime', inplace=True)

            merged = id_df.join(date_df, how='outer',\
                                on='GlucoseDisplayTime', sort=True)

            merged['IsFilledIn'] = 0
            merged.loc[merged.Value.isna(), 'IsFilledIn'] = 1        
            merged.loc[merged.Value.isna(), 'Value'] = mean

            merged['NumId'] = idx

            merged.reset_index(inplace=True)

            merged = merged.drop(columns=['index'])

            merged['TimeLag'] = np.concatenate((merged['GlucoseDisplayTime'].iloc[0],\
                                                np.array(merged['GlucoseDisplayTime'].iloc[:-1].values)), axis=None)\
                                .astype('datetime64[ns]')

            merged['Diff'] = (merged['TimeLag'] - merged['GlucoseDisplayTime']).dt.seconds

            len_merged = len(merged)

            # get all index of rows with diff less than 5 mins, add 1 to remove next row, 
            # dont include last row to delete
            indexes_to_remove = [x for x in merged[merged['Diff'] < 300].index + 1 if x < len_merged & x != 0]

            if len(indexes_to_remove) > 0:
                merged = merged.drop(indexes_to_remove)

            # its ready freddy for some interpoletty
            # merged DF is the dataframe ready to go into interpolation function

            # fill with mean

            merged = merged.drop(columns=['TimeLag', 'Diff'])

            if ((index_counter % 25 != 0) and index_counter != last_idx) or (index_counter == 0):
                merge_df = pd.concat([merge_df, merged])
            elif (index_counter % 25 == 0) or (index_counter == last_idx):
                merge_df = merge_df.astype({'GlucoseDisplayTime': 'datetime64[ns]'})
                
                merge_df.to_parquet('/cephfs/interpolation/' + data_types[j] + '/parquet_' + str(path_counter) + '_' + str(index_counter) + '.parquet')
                merge_df = pd.DataFrame(columns=['GlucoseDisplayTime', 'NumId'])
               
            
            index_counter += 1

        path_counter += 1
    
    print(data_types[j] + ' done')
        

train done
test done
val done


testing if it worked

In [9]:
train_sample = spark.read.format('parquet').load('/cephfs/interpolation/train/parquet_0_25.parquet')

In [11]:
test_sample = spark.read.format('parquet').load('/cephfs/interpolation/test/parquet_0_25.parquet')

In [14]:
val_sample = spark.read.format('parquet').load('/cephfs/interpolation/test/parquet_0_25.parquet')

In [30]:
allPaths = [str(x) for x in list(pathlib.Path('/cephfs/summary_stats/test/').glob('*.parquet'))]

len(allPaths)

404

In [31]:
df = spark.read.format('parquet').load(allPaths)

In [32]:
df.columns

['NumId',
 'Chunk',
 'Mean',
 'StdDev',
 'Median',
 'Min',
 'Max',
 'AvgFirstDiff',
 'AvgSecDiff',
 'StdFirstDiff',
 'StdSecDiff',
 'CountAbove',
 'CountBelow',
 'TotalOutOfRange',
 'target']

In [33]:
df = df.withColumnRenamed('target', 'DiffPrevious')

In [34]:
df = df.withColumn('target', when(df.DiffPrevious > 0, 1)
                                 .when(df.DiffPrevious < 0,-1)
                                 .otherwise(0))

In [35]:
df.select(col('DiffPrevious'), col('target')).show(20)

+------------+------+
|DiffPrevious|target|
+------------+------+
|          77|     1|
|         -28|    -1|
|         -29|    -1|
|          45|     1|
|         -42|    -1|
|          47|     1|
|         -22|    -1|
|           6|     1|
|           2|     1|
|          20|     1|
|           1|     1|
|           0|     0|
|          -9|    -1|
|           8|     1|
|         -30|    -1|
|          23|     1|
|         -41|    -1|
|          49|     1|
|           0|     0|
|         -26|    -1|
+------------+------+
only showing top 20 rows



In [36]:
df.repartition('NumId').write.parquet('/cephfs/summary_stats/test_cat/')

In [37]:
df.dtypes

[('NumId', 'int'),
 ('Chunk', 'int'),
 ('Mean', 'double'),
 ('StdDev', 'double'),
 ('Median', 'float'),
 ('Min', 'float'),
 ('Max', 'float'),
 ('AvgFirstDiff', 'double'),
 ('AvgSecDiff', 'double'),
 ('StdFirstDiff', 'double'),
 ('StdSecDiff', 'double'),
 ('CountAbove', 'bigint'),
 ('CountBelow', 'bigint'),
 ('TotalOutOfRange', 'bigint'),
 ('DiffPrevious', 'bigint'),
 ('target', 'int')]